### Engineering Database Development


In [18]:
import pandas as pd
import pyodbc
import socket
import sqlalchemy.types
from sqlalchemy import create_engine
from urllib import parse
from timeit import default_timer as timer
from datetime import datetime, timedelta

In [19]:
def get_custom_week_number(date):
    """
    Calculate custom week number where week 1 is the first week with at least 4 days
    and weeks start on Sunday.

    Args:
        date: datetime object

    Returns:
        int: Week number for the given date
    """

    year = date.year
    jan_1 = datetime(year, 1, 1)

    # Find what day of week Jan 1 is (0=Monday, 6=Sunday)
    jan_1_weekday = jan_1.weekday()

    # Calculate days from Jan 1 to the next Sunday (start of first full week)
    # If Jan 1 is Sunday (6), days_to_sunday = 0
    # If Jan 1 is Monday (0), days_to_sunday = 6
    days_to_sunday = (6 - jan_1_weekday) % 7

    # First Sunday of the year
    first_sunday = jan_1 + timedelta(days=days_to_sunday)

    # Count how many days are in the partial week before first Sunday
    days_before_first_sunday = days_to_sunday

    # If there are 4+ days before the first Sunday (Thu, Fri, Sat, Sun = 4 days)
    # then Week 1 starts on the Sunday before or on Jan 1
    if days_before_first_sunday >= 4 or jan_1_weekday == 6:
        # Week 1 includes Jan 1
        week_1_start = jan_1 - timedelta(days=(jan_1_weekday + 1) % 7)
    else:
        # Week 1 starts on the first Sunday
        week_1_start = first_sunday

    # If date is before week 1 start, it belongs to last week of previous year
    if date < week_1_start:
        # Recursively get week number from previous year
        prev_year_end = datetime(year - 1, 12, 31)
        return get_custom_week_number(prev_year_end)

    # Calculate week number
    days_since_week_1 = (date - week_1_start).days
    week_number = (days_since_week_1 // 7) + 1

    return week_number


In [20]:
print(get_custom_week_number(datetime.today()))

2


In [2]:
computer_name = socket.gethostname()
print(f"Computer name: {computer_name}")

Computer name: TN-DATACOLLECT2


In [2]:
# Define Database Connections

CONNAS400_PROD = """
Driver={iSeries Access ODBC Driver};
system=10.143.12.10;
Server=AS400;
Database=PROD;
UID=SMY;
PWD=SMY;
"""

CONNAS400_CCSDTA = """
Driver={iSeries Access ODBC Driver};
system=10.143.12.10;
Server=AS400;
Database=CCSDTA;
UID=SMY;
PWD=SMY;
"""

CONNFMA = """
Driver={FileMaker ODBC};
system=10.143.12.196;
Database=mfg8Clamps;
UID=FMODBC;
PWD=FMODBC'
"""

CONNFM = 'DSN=FM Clamp ODBC;UID=FMODBC;PWD=FMODBC'

server = 'tn-sql'
database = 'autodata'
driver = 'ODBC+Driver+17+for+SQL+Server'
user = 'production'
pwd = parse.quote_plus("Auto@matics")
port = '1433'
database_conn = f'mssql+pyodbc://{user}:{pwd}@{server}:{port}/{database}?driver={driver}'
# Make Connection
engine = create_engine(database_conn)
conn_sql = engine.connect()


#### Setup Some SQL Queries

In [9]:
sql_inv = """
SELECT
    STRIP(y.itmid),
    b.qty,
    STRIP(y.itmdesc),
    STRIP(SUBSTR (Altdesc, 15, 1)) Class
FROM
    CCSDTA.DCSCIM y,
    CCSDTA.DMFCMAR x,
    CCSDTA.DCSILM b
WHERE
    x.itmid = y.itmid
    AND x.itmid = b.itmid
    AND x.plt = b.plt
    AND b.plt = '09'
    AND qty <> 0
    AND x.COSTID = 'FRZ'
    AND x.plt NOT IN ('53', '54', '55', '56', '59')
"""

sql_tbl8 = """
    SELECT Ourpart,"Band A Part Number","Housing A Part Number","Screw Part Number","Pack"
    FROM tbl8Tridon
    WHERE  (Ourpart IS NOT NULL)
        AND (RIGHT(Ourpart,1) <> '\r')
        AND ("Housing A Part Number" IS NOT NULL)
        AND (RIGHT("Housing A Part Number",1) <> '\r')
        AND ("Screw Part Number" IS NOT NULL)
        AND (RIGHT("Screw Part Number",1) <> '\r')
        AND ("Band A Part Number" IS NOT NULL)
        AND (RIGHT("Band A Part Number",1) <> '\r')
        AND ("Pack" IS NOT NULL)
        AND (RIGHT("Pack",1) <> '\r')
    ORDER BY Ourpart
    """

In [5]:
def pull_data(conn,qry):
    # Improved connection with proper error handling and connection management
    result = []
    try:
        dbcnxn = pyodbc.connect(conn, timeout=30)
        cursor = dbcnxn.cursor()
        start = timer()

        try:
            cursor.execute(qry)
            result = cursor.fetchall()
            msg = str(len(result)) + " Records Processed From Table"
        except Exception as e:
            msg = conn + ' Query Failed: ' + str(e)
            print(msg)
        finally:
            print(msg)
            print(conn + " Connect/Query Time = " + str(round((timer() - start), 3)) + " sec")

    except pyodbc.Error as e:
        msg = conn + ' Connection Failed: ' + str(e)
        print(msg)
    finally:
        if 'dbcnxn' in locals():
            dbcnxn.close()
    return result

#### Build Dataframe for Component Inventory

In [10]:
df_inv = pd.DataFrame.from_records(pull_data(CONNAS400_CCSDTA,sql_inv))
data_type_dict = {'ITMID': str, 'QTY': int, 'ITMDESC': str, 'CLASS': str}
df_inv.columns = ['ITMID', 'QTY', 'ITMDESC', 'CLASS']
df_inv = df_inv.dropna()
df_inv = df_inv.astype(data_type_dict)
df_inv = df_inv.convert_dtypes()
df_inv.sample(20)

13906 Records Processed From Table

Driver={iSeries Access ODBC Driver};
system=10.143.12.10;
Server=AS400;
Database=CCSDTA;
UID=SMY;
PWD=SMY;
 Connect/Query Time = 0.989 sec


,ITMID,QTY,ITMDESC,CLASS
9806,2306140,2263,BND REG 301SS .022,3
8029,6848051,1370,9/16 HG 201-201-410 SZ 048 1.5-3.5,5
9191,9990584,40,"IDEAL, LARGE DUMP BIN",2
5836,620096706,860,1/2 HG 201-201-305 SZ 096 4.5-6.5,5
6120,6260294,4800,626-02 MIC IND LAB HOME DEP @60(296004),5
10203,238031420052,650,® BORGWARNER UNIVEE T-BOLT BULK@50,5
7357,6706651,370,1/2 HG 316-316-316 SZ 006 0-0,5
1297,1278001,51613,SCR REG 305 SS,3
12763,248420425052,750,® OPTIMAS C-102408 MX BULK@50,5
1735,2013001,1124,ROD .240 CAR 1018 MIDI SCR,1


#### Build Dataframe for Clamp Data

In [27]:

df_clamps = pd.DataFrame.from_records(pull_data(CONNFM,sql_tbl8))
# Set Column Names
df_clamps.columns = ['PartNumber', 'Band', 'Housing', 'Screw', 'Pack']
# Set Data Types
data_type_dict = {'PartNumber': str, 'Band': str, 'Housing': str, 'Screw': str, 'Pack': str}

# Do Some Filtering and Data Cleansing
df_clamps = df_clamps[1:]
df_clamps = df_clamps.astype(data_type_dict)
df_clamps = df_clamps.dropna()
df_clamps = df_clamps.convert_dtypes()
df_clamps.sample(20)



7391 Records Processed From Table
DSN=FM Clamp ODBC;UID=FMODBC;PWD=FMODBC Connect/Query Time = 1.781 sec


,PartNumber,Band,Housing,Screw,Pack
2326,600070092127,2115092,1151007,2323001,127
5207,844700102,2005700,2300001,1220001,102
2616,602048102,2366048,2350011,1213002,102
1927,573140060099,4854060,4856003052,4452012052,099
2027,574040075099,4874075,4876001052,4491012052,099
784,360040024M01,2373024M,1151005,1278001MC,M01
5212,844850000,2005850,2300001,1220001,000
2549,600450800052,2966812,7009900,2543001,052
5402,DM296024000,1074024,4097037,D4211001,000
6996,M74807A102,108607A,2367012,1278001,102


#### Load Inventory Data to SQL Server Table

In [11]:
data_type_dict = {'ITMID' : sqlalchemy.types.VARCHAR(255),'QTY' : sqlalchemy.types.INT,'ITMDESC' : sqlalchemy.types.VARCHAR(255),
                  'CLASS' : sqlalchemy.types.VARCHAR(255)}

df_inv.to_sql('tblCompInv', conn_sql, schema='eng', if_exists='replace', index=False, dtype=data_type_dict)

282

In [16]:
data_type_dict = {'PartNumber' : sqlalchemy.types.VARCHAR(255), 'Band' : sqlalchemy.types.VARCHAR(255),'Housing' : sqlalchemy.types.VARCHAR(255),'Screw' : sqlalchemy.types.VARCHAR(255), 'Pack' : sqlalchemy.types.VARCHAR(255)}

df_clamps.to_sql('tbl8', conn_sql, schema='eng', if_exists='replace', index=False, dtype=data_type_dict)

267